Item 38 Accept Functions Instead of Classes for Simple Interfaces

Things to Remember
- Instead of defining and instantiating classes, you can often simply use functions for simple interfaces between components in Python.
- References to functions and methods in Python are first class, meaning they can be used in expressions (like any other type).
- The \__call\_ special method enables instances of a class to be called like plain Python functions.
- When you need a function to maintain state, consider defining a class that provides the \__call\__ method instead of defining a stateful closure.

Background
- Many of Python's built-in APIs allow you to customize behavior by passing in a function.
- These hooks are used by APIs to call back your code while they execute.
- In Python, many hooks are just stateless functions with well-defined arguments and return values.
- Functions are easier to describe and simpler to define than classes. 

In [ ]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=len)
print(names)

In [ ]:
# customize the defaultdict

def log_missing(): # accepts no arguments
    print('Key added') # your code 
    # - must return the default value
    #   that the missing key should 
    #   have in the dict 
    return 0

In [ ]:
from collections import defaultdict

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]

result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))


- Supplying hooks makes APIs easy to build and test because it separates side effects (your code) from deterministic behavior (the default value returned)

In [5]:
# side effects using a stateful closure (check Item 21)
def increment_with_report(current, increments):
    added_count = 0
    def missing():
        nonlocal added_count # stateful closure
        added_count += 1
        return 0
    # - the defaultdict has no idea that
    #   the missing hook maintain state     
    result = defaultdict(missing, current) 
    for key, amount in increments:
        result[key] += amount
    return result, added_count  

In [6]:
result, count = increment_with_report(current, increments)
assert count == 2

In [7]:
# - define a small class that encapsulates
#   the state
# - it's clearer than the stateful closure approach
class CountMissing:
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

In [8]:
counter = CountMissing()
# - pass the 'missing' method as
#   the default value hook
result = defaultdict(counter.missing, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

In [9]:
# - define the __call__ special method
# - this allows an object to be called
#   just like a function

class BetterCountMissing:
    def __init__(self):
        self.added = 0
    # - indicates that a class's instances will be used
    #   somewhere a function argument would be suitable
    #   (like API hooks)

    def __call__(self):
        self.added += 1
        return 0  

In [12]:
counter = BetterCountMissing()
assert counter() == 0 # call the object like a function
assert callable(counter) # True

In [13]:
counter = BetterCountMissing()
result = defaultdict(counter, current) # relies on __call__
for key, amount in increments:
    result[key] += amount
assert counter.added == 2 